In [19]:
import pickle
import pandas as pd 

from lib import policies
import numpy as np

In [53]:
# actions: hit or stand
ACTION_HIT = 0
ACTION_STAND = 1  #  "strike" 
ACTIONS = [ACTION_HIT, ACTION_STAND]

def banchmark_policy():
    def policy_fn(state,Q_vals ):
        player_sum =state[0]
        return ACTION_HIT if player_sum<17 else ACTION_STAND
    return policy_fn


def dealer_policy(dealer_sum):
    return ACTION_HIT if dealer_sum<17 else ACTION_STAND
# get a new card
def get_card():
    card = np.random.randint(1, 14)
    return  card ,11 if card == 1 else min(card, 10)


def load( name):
    Q_vals = pd.read_pickle(name)
    return Q_vals

def model_Q_vals( action_size):

    def policy_fn(state,Q_vals  ): 
#         print( state )
#         print( list(Q_vals[ state ].values()) )
        best_action = np.argmax(list(Q_vals[ state ].values()))
#         print(best_action)
        return best_action
    
    return policy_fn

def make_epsilon_greedy_policy(estimator, nA):

    def policy_fn(  state,Q_vals):
        s =[]
        s.append(  state)  
        q_values = estimator.predict(np.array(s )) 
        best_action = np.argmax(q_values)
        
        return best_action
    return policy_fn


def play( player_sum, dealer_sum,player_ace,policy_player, Q_vals=None ):
 
   
    #print(dealer_cards['card_name'].iloc[0] )
    #print(dealer_cards)
    #player_ace =    player_cards[player_cards.card_name== 1]
    
    
    #state = [usable_ace_player, player_sum, dealer_card1]

    
#     banchmark_reward = 0
    player_reward =0
    
#     banchmark_player_sum = player_sum
    # player's turn
    while True:
        # get action based on current sum
        state = (player_sum, dealer_sum,player_ace)
        action = policy_player(state,Q_vals)

         
        if action == ACTION_STAND:
            break
        # if hit, get new card
        card, new_card_value = get_card()
         
        
        # Keep track of the ace count. the usable_ace_player flag is insufficient alone as it cannot
        # distinguish between having one ace or two.
        if card==1:
            player_ace=True
            if (player_sum+new_card_value) > 21:
                new_card_value=1
        
        player_sum = player_sum+new_card_value
#         if banchmark_player_sum<17:
#             banchmark_player_sum=player_sum
            

#         # player busts
        if player_sum > 21:
            player_reward= -1
            break
 
        

    # dealer's turn
    while True:
        # get action based on current sum
        action = dealer_policy(dealer_sum)
        
        if action == ACTION_STAND:
            break
            
        # if hit, get a new card     
        dealer_card, new_card_value  = get_card()
        #new_card = {'card_name':dealer_card, 'value':new_card_value}
        #dealer_cards = dealer_cards.append(new_card, ignore_index=True)
        if dealer_card==1:
            ace = True#   dealer_cards[dealer_cards.card_name== 1] 
        
        # If the dealer has a usable ace, use it as 1 to avoid busting and continue.
        if dealer_card==1:
            if (dealer_sum + new_card_value) > 21 and ace== True:
                new_card_value=1
        
        dealer_sum = dealer_sum + new_card_value
        # dealer busts
        #dealer_sum = sum(dealer_cards['value'])
        if dealer_sum > 21:
            player_reward=1
 
            return  player_reward  

    
    # Reward Calculation
#     print("P:{},D:{},B:{}".format(player_sum,dealer_sum,banchmark_player_sum))
#     print("d:{}".format(dealer_sum))
#     assert player_sum <= 21 and dealer_sum <= 21
    if player_reward!=-1:
        if player_sum > dealer_sum:
            player_reward=  1 
        elif player_sum == dealer_sum:
            player_reward=  0 
        else:
            player_reward=  -1 

#     if banchmark_player_sum > dealer_sum:
#         banchmark_reward=  1 
#     elif banchmark_player_sum == dealer_sum:
#         banchmark_reward=  0 
#     else:
#         banchmark_reward=  -1 
    
    return  player_reward 

In [24]:
#final_MC_off_vals_50K

In [25]:
all_states = [(p_sum, d_sum, ace) for ace in  (True,False) for p_sum in range(12,22) for d_sum in range(1,13)]
#all_states

# len(all_states)

In [27]:
# Q=load('model_output/final_MC_off_vals_100K.pkl')
# r=play(19, 7, False, current_policy,Q)
# r

In [31]:
def run_simulation(Q,name):
    win=0
    loss=0
    draw=0



    b_win=0
    b_loss=0
    b_draw=0

    np.random.seed(1234)
    
    for st in all_states:
        player_reward=play(st[0], st[1], st[2], current_policy,Q)
        if player_reward==1:
            win=win+1
        elif player_reward==-1:
            loss=loss+1
        else:
            draw=draw+1

#         if banchmark_reward==1:
#             b_win=b_win+1
#         elif banchmark_reward==-1:
#             b_loss=b_loss+1
#         else:
#             b_draw=b_draw+1
    print("Name:{}---Agent---win:{},loss:{}, draw:{}".format(name, win, loss, draw))
#     print("Name:{}---Benchmark---win:{},loss:{}, draw:{}".format(name, b_win, b_loss, b_draw))
    
    return win/len(all_states),b_win/len(all_states)


# Banchmark

In [33]:
current_policy= banchmark_policy(  )
run_simulation(None,'banchmark')

Name:banchmark---Agent---win:123,loss:96, draw:21


(0.5125, 0.0)

# MC_off

In [34]:
current_policy= model_Q_vals( 2 )

In [35]:
Q=load('model_output/final_MC_off_vals_50K.pkl')
run_simulation(Q,'MC_off')
 

Name:MC_off---Agent---win:127,loss:95, draw:18


(0.5291666666666667, 0.0)

# DynaQ

In [43]:
Q=load('model_output/final_DynaQ_vals.pkl')
run_simulation(Q,'DynaQ')


Name:DynaQ---Agent---win:117,loss:93, draw:30


(0.4875, 0.0)

# QLearning

In [37]:
Q=load('model_output/final_QLearning_50K.pkl')
run_simulation(Q,'QLearning')

Name:QLearning---Agent---win:121,loss:109, draw:10


(0.5041666666666667, 0.0)

# DQN

In [54]:
from agents import dqn

In [55]:
agent = dqn.DQNAgent(500,32,"model_output/",False)

In [56]:
agent.load("model_output/final_weights.hdf5")

In [57]:
current_policy=make_epsilon_greedy_policy(agent.q_estimator,2)
run_simulation(None,'DQN')
 

Name:DQN---Agent---win:117,loss:93, draw:30


(0.4875, 0.0)

In [17]:
np.random.seed(1234)
np.random.randint(1, 14)

4